# Importation des bibliotheques

In [15]:
import os
import re
from collections import defaultdict
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from timm import create_model
import time 
import json


In [2]:
# Chemin du dossier train_data
dossier_train_data = r"C:\Users\Christian\Desktop\UE Projet\Dataset\train_data"

# Dictionnaire pour stocker le nombre d'images par classe
nombre_images_par_classe = defaultdict(int)

# Parcourir tous les fichiers dans le dossier train_data
for file_name in os.listdir(dossier_train_data):
    """Les fichiers sont dans un dossiers train data déja prétraités. Nous allons extraire le nombre qui se trouve dans la structure du nom de chaque fichiers et conserver le nom de l'image qui represente la classe"""
    # Utiliser une expression régulière pour extraire la partie avant le nombre
    match = re.match(r"^(.*?)(\d+)\.jpeg$", file_name)
    if match:
        class_name = match.group(1)
        nombre_images_par_classe[class_name] += 1

# Afficher les résultats
print("Nombre d'images par classe dans train_data :")
for classe, nombre in nombre_images_par_classe.items():
    print(f"{classe} : {nombre}")


Nombre d'images par classe dans train_data :


In [3]:
"""Ici on va stocker les images par classes pour voir la classe a laquelle chaque image correspond"""
# Dictionnaire pour stocker les fichiers par classe
fichiers_par_classe = {}

# Parcourir tous les fichiers dans le dossier train_data
for file_name in os.listdir(dossier_train_data):
    # Utiliser une expression régulière pour extraire la classe
    match = re.match(r"^(.*?)(\d+)\.jpeg$", file_name)
    if match:
        class_name = match.group(1)
        if class_name not in fichiers_par_classe:
            fichiers_par_classe[class_name] = []
        fichiers_par_classe[class_name].append(file_name)
        

In [4]:
# Ici on affiche juste les images par classe.
fichiers_par_classe

{}

### Ici on va entrainer notre modele sur notre dataset d'entrainement complet

In [5]:
# Chemin du dossier train_data
dossier_train_data = r"C:\Users\Christian\Desktop\UE Projet\Dataset\train_data"

# Organiser les images en sous-dossiers
for file_name in os.listdir(dossier_train_data):
    # Utiliser une expression régulière pour extraire la classe
    match = re.match(r"^(.*?)(\d+)\.jpeg$", file_name)
    if match:
        class_name = match.group(1)
        class_dir = os.path.join(dossier_train_data, class_name)
        
        # Créer le sous-dossier si nécessaire
        if not os.path.exists(class_dir):
            os.makedirs(class_dir)
        
        # Déplacer l'image dans le sous-dossier
        src_path = os.path.join(dossier_train_data, file_name)
        dst_path = os.path.join(class_dir, file_name)
        shutil.move(src_path, dst_path)

print("Images réorganisées en sous-dossiers par classe.")


Images réorganisées en sous-dossiers par classe.


In [6]:
# Les données d'entrainement et de test
dossier_train = r"C:\Users\Christian\Desktop\UE Projet\Dataset\train_data"

# Définitions des paramètres pour la taille des images et le nombres d'images traités par lot
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

# Créer un dataset à partir des fichiers d'entraînement
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory=dossier_train,
    labels='inferred', # Pour specifier que chaque images de chaque sous repertoires appartient uniquement a cette classe
    label_mode='int', # Encodage en entier pour convertir les labels(classes/sous repertoire) en entier afin de faciliter l'entrainement
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    shuffle=True
)

# Normaliser les images
normalization_layer = tf.keras.layers.Rescaling(1./255)

# Appliquer la normalisation aux datasets
train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))

# Afficher la structure des datasets
for images, labels in train_dataset.take(1):
    print(images.shape)  # Afficher la forme des tenseurs d'images
    print(labels.shape)  # Afficher la forme des tenseurs de labels


Found 3111 files belonging to 137 classes.
(32, 128, 128, 3)
(32,)


### Application de la data augmentation

Grace a cette technique, lors de l'entrainement du modele sur 10 epoques, le modele aura un equivalent d'environ 30000 images dans sa base d'entrainement. Le principe est le suivant: Lors de l'entrainement du modele sur chaque époques, chaque image recevra les transformations suivante avant d'etre reconnu par le modele comme element d'une classe.

In [7]:
# Chemin des dossiers train et test
dossier_train = r"C:\Users\Christian\Desktop\UE Projet\Dataset\train_data"

# Créer un générateur d'images avec augmentation pour l'entraînement
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Appliquer le générateur aux données d'entraînement
train_generator = train_datagen.flow_from_directory(
    directory=dossier_train,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='sparse'  # Utiliser 'sparse' pour les étiquettes entières
)

# Afficher la structure des datasets
for images, labels in train_generator:
    print(images.shape)  # Afficher la forme des tenseurs d'images
    print(labels.shape)  # Afficher la forme des tenseurs de labels
    break  # Pour éviter d'afficher toutes les images


Found 3111 images belonging to 137 classes.
(32, 128, 128, 3)
(32,)


### Vérifieons le contenu du répertoire train avant son passage dans le modele et voyons a quoi correspond chaque classe de l'ensemble d'entrainement par rapport aux classes de depart. On va verifier cela avec les indices car plus haut nous avons utiliser un encodage pour encoder nos differentes classes en valeurs entieres (le target encoding) pour faciliter le passage dans le modele


In [8]:
# Chemin du dossier d'entrainement
dossier_train = r"C:\Users\Christian\Desktop\UE Projet\Dataset\train_data"
# Affichage des classes
print("Contenu du répertoire 'train_data' :")
for file_name in os.listdir(dossier_train):
    print(file_name)


Contenu du répertoire 'train_data' :
Actiniaria_
Actinoptilum_molle_
Actinoscyphia_plebeia_
Actinostola_capensis_
Aequorea_spp_
Africolaria_rutila_
Alcyonacea_
Amalda_bullioides_
Anthoptilum_grandiflorum_
Aphelodoris_sp__
Aphrodita_alta_
Aristeus_varidens_
Armina_sp__
Ascidiacea_
Astropecten_irregularis_pontoporeus_
Athleta_abyssicola_
Athleta_lutosa_
Bolocera_kerguelensis_
Brissopsis_lyrifera_capensis_
Bryozoa_
Cavernularia_spp_
Cephalodiscus_gilchristi_
Ceramaster_patagonicus_euryplax_
Charonia_lampas_
Cheilostomatida_
Cheiraster_hirsutus_
Chondraster_elattosis_
Chrysaora_fulgida_
Chrysaora_spp_
Comanthus_wahlbergii_
Comitas_saldanhae_
Comitas_stolida_
Cosmasterias_felipes_
Crossaster_penicillatus_
Cypraeovula_iutsui_
Diplopteraster_multipes_
Dipsacaster_sladeni_capensis_
Echinus_gilchristi_
Eleutherobia_variable_
Euspira_napus_
Exodromidia_spinosa_
Exodromidia_spinosissima_
Flabellum_(Ulocyathus)_messum_
Funchalia_woodwardi_
Fusinus_africanae_
Fusinus_hayesi_
Fusitriton_magellanicus

In [9]:
# Récupérer les noms des classes
class_names = train_generator.class_indices
class_names = {v: k for k, v in class_names.items()}  # Inverser le dictionnaire pour obtenir les noms des classes

# Affichage des noms des classes 
print("Classes et indices correspondants :")
for idx, class_name in class_names.items():
    print(f"Indice {idx} : {class_name}")


Classes et indices correspondants :
Indice 0 : Actiniaria_
Indice 1 : Actinoptilum_molle_
Indice 2 : Actinoscyphia_plebeia_
Indice 3 : Actinostola_capensis_
Indice 4 : Aequorea_spp_
Indice 5 : Africolaria_rutila_
Indice 6 : Alcyonacea_
Indice 7 : Amalda_bullioides_
Indice 8 : Anthoptilum_grandiflorum_
Indice 9 : Aphelodoris_sp__
Indice 10 : Aphrodita_alta_
Indice 11 : Aristeus_varidens_
Indice 12 : Armina_sp__
Indice 13 : Ascidiacea_
Indice 14 : Astropecten_irregularis_pontoporeus_
Indice 15 : Athleta_abyssicola_
Indice 16 : Athleta_lutosa_
Indice 17 : Bolocera_kerguelensis_
Indice 18 : Brissopsis_lyrifera_capensis_
Indice 19 : Bryozoa_
Indice 20 : Cavernularia_spp_
Indice 21 : Cephalodiscus_gilchristi_
Indice 22 : Ceramaster_patagonicus_euryplax_
Indice 23 : Charonia_lampas_
Indice 24 : Cheilostomatida_
Indice 25 : Cheiraster_hirsutus_
Indice 26 : Chondraster_elattosis_
Indice 27 : Chrysaora_fulgida_
Indice 28 : Chrysaora_spp_
Indice 29 : Comanthus_wahlbergii_
Indice 30 : Comitas_sald

In [10]:
# Chemin du dataset
data_dir = r'C:\Users\Christian\Desktop\UE Projet\Dataset\train_data'

# Préparer les transformations et le DataLoader
transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.ToTensor(), # Ici on normalise les images en les fesant passer de la plage [0,255] en la plage [0,1] car initialement les images sont des tenseurs numpy
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # Normalisation appliqué au format des couleurs RGB pour une moyenne de mean et un ecart type de std
]) # On aurait pu ajouter d'autre transformation mais ca serait une redondance car cela a deja été fait dans notre section data augmentation.

# Application de ce second prétraitement aux données de la base train_data
train_dataset = datasets.ImageFolder(root=data_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Charger le modèle ViT pré-entraîné depuis timm avec 137 sorties correspondant au 137 classes
model = create_model('vit_base_patch16_224', pretrained=True)
model.head = nn.Linear(model.head.in_features, 137)

# Définir l'optimiseur et la fonction de perte
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


In [12]:
# Entrainement du model avec affichage de la précision et de la perte
for epoch in range(5):  
    start_time = time.time() # Début de l'époque
    correct_train = 0 # Nombre total d'image correctement predit dans cette epoque
    total_train = 0 # Nombre total d'image vu dans cette epoque
    for images, labels in train_loader:
        optimizer.zero_grad() # Reinitialisation des gradients
        outputs = model(images) # Predictions
        loss = criterion(outputs, labels) # Loss
        loss.backward() # Retropropagation
        optimizer.step() # Ajustement des parametres
        
        _, predicted = torch.max(outputs.data, 1) # On reccupere l'indice de la classe predite avec la plus grande probabilité
        total_train += labels.size(0) # Mise a jour du nombre d'image vu dans cette epoque jusqu'a cette etapes
        correct_train += (predicted == labels).sum().item() # Mise a jour du Nombre total d'image correctement predit dans cette epoque
    
    train_accuracy = 100 * correct_train / total_train # Calcul de la precision apres chaque etapes
    end_time = time.time() # Fin de l'époque 
    epoch_time = end_time - start_time # Calcul du temps d'exécution 
    print(f"Epoch [{epoch+1}/10], Loss: {loss.item():.4f}, Train Accuracy: {train_accuracy:.2f}%, Time: {epoch_time:.2f} seconds")
    #print(f"Epoch [{epoch+1}/10], Loss: {loss.item():.4f}, Train Accuracy: {train_accuracy:.2f}%") # affichage de l'epoque, de la perte et de la precision


Epoch [1/10], Loss: 0.1150, Train Accuracy: 93.99%, Time: 9689.41 seconds
Epoch [2/10], Loss: 0.0107, Train Accuracy: 96.01%, Time: 289364.29 seconds
Epoch [3/10], Loss: 0.0005, Train Accuracy: 98.23%, Time: 3876.70 seconds
Epoch [4/10], Loss: 0.0725, Train Accuracy: 97.94%, Time: 5188.05 seconds
Epoch [5/10], Loss: 0.0003, Train Accuracy: 99.45%, Time: 4061.54 seconds


In [14]:
torch.save(model.state_dict(), "model.pth")

In [16]:
class_indices = train_generator.class_indices
with open("class_indices.json", "w") as f:
    json.dump(class_indices, f)